# Продвинутый Python, семинар 6

**Лектор:** Петров Тимур

**Семинаристы:** Садуллаев Музаффар, Бузаев Федор, Дешеулин Олег, Коган Александра, Васина Олеся

**Spoiler Alert:** в рамках курса нельзя изучить ни одну из тем от и до досконально (к сожалению, на это требуется больше времени, чем даже 3 часа в неделю). Но мы попробуем рассказать столько, сколько возможно :)

![](https://www.sqlitetutorial.net/wp-content/uploads/2015/11/sqlite-sample-database-color.jpg)

## Работаем с ORM

Выглядит это вот так:

![](https://s3.amazonaws.com/media-p.slid.es/uploads/10882/images/5116660/sqlalchemy__3_.png)

* DB API - все, что про cursor, connection etc

* Core - это похожее на Query Builder структура, возможность создавать запросы

* ORM - уже модуль для маппинга результатов к объектам

Установим нужные зависимости

In [ ]:
!pip install pypika
!pip install sqlalchemy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pypika: filename=PyPika-0.48.9-py2.py3-none-any.whl size=53723 sha256=04d79d762b0cfd78ce61cc43b3dd95fe8ea2b5d348b39476430f420458f02ade
  Stored in directory: /root/.cache/pip/wheels/e1/26/51/d0bffb3d2fd82256676d7ad3003faea3bd6dddc9577af665f4
Successfully built pypika


Прежде чем мы начнем делать SQL-запросы давайте подготовим сущности над объектами БД

In [ ]:
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey, Table, Float
from sqlalchemy.orm import relationship, declarative_base, sessionmaker

In [ ]:
Base = declarative_base()

In [ ]:
class Playlist(Base):
    __table_args__ = {'extend_existing': True}
    __tablename__ = 'playlists'

    PlaylistId = Column(Integer, primary_key=True)
    Name = Column(String)

    tracks = relationship('Track', secondary='playlist_track', back_populates='playlists')

Давайте поподробнее что мы тут сделали:

Здесь мы указали сущность плейлиста. Почему это называется сущностью?

Оно отнаследовалось от класса Base. В контексте ORM все сущности наследуются от него. К тому же мы указали название таблицы в котором оно определяется.

Так как у таблиц есть отношения с другими таблицами также мы определили `relationship` с другой таблицей.

В текущем примере мы создали отношение `Many-To-Many` (см. первую картинку) которое реализовано через промежуточную таблицу `playlist_track`. Это означает, что один плейлист может содержать много треков, а один трек может включен в несколько плейлистов.

Сделаем такое же для остальных сущностей

In [ ]:
class Genre(Base):
    __tablename__ = 'genres'

    GenreId = Column(Integer, primary_key=True)
    Name = Column(String)

    tracks = relationship('Track', back_populates='genre')

In [ ]:
class Track(Base):
    __tablename__ = 'tracks'

    TrackId = Column(Integer, primary_key=True)
    Name = Column(String)
    AlbumId = Column(Integer, ForeignKey('albums.AlbumId'))
    GenreId = Column(Integer, ForeignKey('genres.GenreId'))
    Composer = Column(String)
    Milliseconds = Column(Integer)
    Bytes = Column(Integer)
    UnitPrice = Column(Float)

    genre = relationship('Genre', back_populates='tracks')
    album = relationship('Album', back_populates='tracks')
    playlists = relationship('Playlist', secondary='playlist_track', back_populates='tracks')
    invoice_items = relationship('InvoiceItem', back_populates='track')

In [ ]:
class Album(Base):
    __tablename__ = 'albums'

    AlbumId = Column(Integer, primary_key=True)
    Title = Column(String)
    ArtistId = Column(Integer, ForeignKey('artists.ArtistId'))

    artist = relationship('Artist', back_populates='albums')
    tracks = relationship('Track', back_populates='album')

In [ ]:
class Artist(Base):
    __tablename__ = 'artists'

    ArtistId = Column(Integer, primary_key=True)
    Name = Column(String)

    albums = relationship('Album', back_populates='artist')

In [ ]:
class InvoiceItem(Base):
    __tablename__ = 'invoice_items'

    InvoiceLineId = Column(Integer, primary_key=True)
    InvoiceId = Column(Integer, ForeignKey('invoices.InvoiceId'))
    TrackId = Column(Integer, ForeignKey('tracks.TrackId'))
    UnitPrice = Column(Float)
    Quantity = Column(Integer)

    track = relationship('Track', back_populates='invoice_items')
    invoice = relationship('Invoice', back_populates='invoice_items')

In [ ]:
class Invoice(Base):
    __tablename__ = 'invoices'

    InvoiceId = Column(Integer, primary_key=True)
    CustomerId = Column(Integer, ForeignKey('customers.CustomerId'))
    Total = Column(Float)

    customer = relationship('Customer', back_populates='invoices')
    invoice_items = relationship('InvoiceItem', back_populates='invoice')

In [ ]:
class Customer(Base):
    __tablename__ = 'customers'

    CustomerId = Column(Integer, primary_key=True)
    FirstName = Column(String)
    LastName = Column(String)
    Email = Column(String)
    Address = Column(String)
    City = Column(String)
    State = Column(String)
    Country = Column(String)
    PostalCode = Column(String)

    invoices = relationship('Invoice', back_populates='customer')

In [ ]:
class PlaylistTrack(Base):
    __table_args__ = {'extend_existing': True}
    __tablename__ = 'playlist_track'

    PlaylistId = Column(Integer, ForeignKey('playlists.PlaylistId'), primary_key=True)
    TrackId = Column(Integer, ForeignKey('tracks.TrackId'), primary_key=True)

Теперь создадим сессию:

In [ ]:
!wget https://github.com/Palladain/Deep_Python_2023/raw/main/week07/chinook.db

--2024-10-13 20:43:57--  https://github.com/Palladain/Deep_Python_2023/raw/main/week07/chinook.db
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Palladain/Deep_Python_2023/main/week07/chinook.db [following]
--2024-10-13 20:43:57--  https://raw.githubusercontent.com/Palladain/Deep_Python_2023/main/week07/chinook.db
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 884736 (864K) [application/octet-stream]
Saving to: ‘chinook.db’

chinook.db          100%[===================>] 864.00K  --.-KB/s    in 0.08s   

2024-10-13 20:43:58 (11.0 MB/s) - ‘chinook.db’ saved [884736/884736]



In [ ]:
engine = create_engine('sqlite:///chinook.db')
Session = sessionmaker(bind=engine)
session = Session()

Сессия это ключевой компонент, который управляет взаимодействием с базой данных.

В задачи сессии входит:
* Соединение с базой данных
* Управление объектами
* Изменение состояния объектов

Сделаем запрос:

In [ ]:
test_query = session.query(Playlist.Name).all()

print(test_query)

NameError: name 'Playlist' is not defined

Как видим мы делаем запрос в БД с помощью кода, написав 0 строчек SQL

**Задание** Подсчитать количество треков в каждом плейлисте. Необходимо вывести в паттерне `{"name_list": "NAME_LIST", "num_tracks": 0}`

In [ ]:
from sqlalchemy import func ## Необходимые функции для агрегации

playlist_counts = #YOUR CODE

<ipython-input-17-7085945bc717>:5: SAWarning: SELECT statement has a cartesian product between FROM element(s) "playlist_track" and FROM element "playlists".  Apply join condition(s) between each element to resolve.
  ).group_by(Playlist.Name).all()


In [ ]:
for playlist, count in playlist_counts:
    print(f"list = {playlist}, count = {count}")

**Задание** Получить все альбомы и их треки

In [ ]:
# YOUR CODE

**Задание**  Найти треки без привязанных альбомов

In [ ]:
#YOUR CODE

**Задание** Найти все жанры и количество треков в каждом жанре

In [ ]:
from sqlalchemy import func

#YOUR CODE

**Задание** Найти плейлисты, которые не содержат ни одного трека

In [ ]:
# YOUR CODE

Playlist without tracks: Movies
Playlist without tracks: Audiobooks
Playlist without tracks: Audiobooks
Playlist without tracks: Movies


**Задание** Получить всех клиентов и их счета

In [ ]:
#YOUR CODE

**Задание** Найти все треки с их альбомами

In [ ]:
from sqlalchemy import func

#YOUR CODE

Плейлист не найден.


### Каcкадные операции

**на этом шаге ipynb необходимо перезапустить!**

Каскадные операции в алхимии позволяют автоматически выполнять действия с зависимыми объектами при изменении или удалении основного объекта. \
Например, можно автоматически удалить все связанные с объектом записи, если основной объект удаляется.

Давайте настроим это!

In [ ]:
from sqlalchemy import Column, Integer, String, ForeignKey, create_engine
from sqlalchemy.orm import relationship, sessionmaker
from sqlalchemy.ext.declarative import declarative_base

In [ ]:
Base = declarative_base()

/var/folders/1k/s7xycc896gd2kbcgp89z21_06b4zqs/T/ipykernel_94246/4196137762.py:1: MovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [ ]:
class Artist(Base):
    __tablename__ = 'artists'

    ArtistId = Column(Integer, primary_key=True)
    Name = Column(String)

    albums = relationship('Album', back_populates='artist', cascade='all, delete')

In [ ]:
class Album(Base):
    __tablename__ = 'albums'

    AlbumId = Column(Integer, primary_key=True)
    Title = Column(String)
    ArtistId = Column(Integer, ForeignKey('artists.ArtistId'))

    artist = relationship('Artist', back_populates='albums')

In [ ]:
engine = create_engine('sqlite:///chinook.db')
Session = sessionmaker(bind=engine)
session = Session()

In [ ]:
new_artist = Artist(Name="Артист Артистович")
new_album1 = Album(Title="Первый альбом", artist=new_artist)
new_album2 = Album(Title="Второй альбом", artist=new_artist)

In [ ]:
session.add(new_artist)
session.commit()

In [ ]:
print("Добавили артиста и его альбомы!")
artist = session.query(Artist).filter_by(Name="Артист Артистович").first()
print(f"Новый артист: {artist.Name}")
for album in artist.albums:
    print(f"\t- Альбомы: {album.Title}")

Добавили артиста и его альбомы!
Новый артист: Артист Артистович
	- Альбомы: Первый альбом
	- Альбомы: Второй альбом


In [ ]:
# откатим назад
session.delete(artist)
session.commit()

In [ ]:
deleted_artist = session.query(Artist).filter_by(Name="Артист Артистович").first()
deleted_albums = session.query(Album).filter(Album.Title.in_(["Первый альбом", "Второй альбом"])).all()

print("Артист:", deleted_artist)
print("Его альбомы: ", deleted_albums)

Артист: None
Его альбомы:  []


### А что если генерировать ORM модели?

(запускайте это локально!!!)

Такое в природе существует. Используется в основном для дебага или по другим причинам когда не хочется писать свои модели. Установим библиотеку.

In [ ]:
!pip install sqlacodegen

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.1 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.35
    Uninstalling SQLAlchemy-2.0.35:
      Successfully uninstalled SQLAlchemy-2.0.35
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.54 which is incompatible.


In [ ]:
!sqlacodegen sqlite:///chinook.db --outfile models.py

In [ ]:
from models import Artist

new_artist = Artist(Name="Артист Артистович")
session.add(new_artist)


artist = session.query(Artist).filter_by(Name="Артист Артистович").first()
print("Найдет артист:", artist)